In [1]:
import os

In [2]:
%pwd

'c:\\Users\\Ivar\\Downloads\\text summarzre\\text-summarizer\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'c:\\Users\\Ivar\\Downloads\\text summarzre\\text-summarizer'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs:int
    warmup_steps  :int
    per_device_train_batch_size  :int
    weight_decay:int
    logging_steps:int
    evaluation_strategy:int
    eval_steps:int
    save_steps:int
    gradient_accumulation_steps:int
    
    

In [6]:
from textsummarzer.constants import CONFIG_FILE_PATH,CONFIG_FILE_PATH,PARAMS_FILE_PATH
from textsummarzer.utils.common import read_yaml,create_directory

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        
        create_directory([self.config.artifacts_root]) 
    def get_data_trainer_config(self)->ModelTrainConfig:
      
      config=self.config.model_trainer
      params=self.params.TrainingArguments
      create_directory([config.root_dir])
      
      data_trainer_config=ModelTrainConfig (
        root_dir=config.root_dir ,
        data_path=config.data_path,
        model_ckpt=config.model_ckpt,
        num_train_epochs=params.num_train_epochs,
        warmup_steps  = params. warmup_steps ,
        per_device_train_batch_size  = params.per_device_train_batch_size  ,
        weight_decay=params.weight_decay,
        logging_steps=params.logging_steps,
        evaluation_strategy= params.evaluation_strategy,
        eval_steps= params.eval_steps,
        save_steps=params.save_steps,
        gradient_accumulation_steps= params.gradient_accumulation_steps)
      
      return data_trainer_config

In [8]:
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from datasets import load_dataset,load_from_disk
import torch 

class ModelTrainer:
    def __init__(self,config:ModelTrainConfig):
        self.config=config
    def train(self):
        model_ckpt = "google/pegasus-cnn_dailymail"
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)
            
            #loading data 
        dataset_samsum_pt = load_from_disk(self.config.data_path)
        
        trainer_args = TrainingArguments(
            output_dir='pegasus-samsum', num_train_epochs=1, warmup_steps=500,
            per_device_train_batch_size=1, per_device_eval_batch_size=1,
            weight_decay=0.01, logging_steps=10,
            evaluation_strategy='steps', eval_steps=500, save_steps=1e6,
            gradient_accumulation_steps=16)
        trainer = Trainer(model=model_pegasus, args=trainer_args,
                        tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                        train_dataset=dataset_samsum_pt["test"],
                        eval_dataset=dataset_samsum_pt["validation"])
        trainer.train()
        model_pegasus.save_pretrained(os.path.join(self.config.root_dir,'pegasus-samsum'))
        tokenizer.save_pretrained(os.path.join(self.config.root_dir,'tokenizer-samsum'))



c:\Users\Ivar\Downloads\text summarzre\text-summarizer\texts\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2024-10-20 20:51:11,585:INFO:config:PyTorch version 2.4.1 available.]


In [9]:
#create the pipline
try:
    config=ConfigurationManager()
    DataTrainConfig=config.get_data_trainer_config()
    DataTrainConfig=ModelTrainer(config=DataTrainConfig)
    DataTrainConfig.train()

except Exception as e:
    raise e

[2024-10-20 20:51:14,113:INFO:common:yaml file: C:\Users\Ivar\Downloads\text summarzre\text-summarizer\config\config.yaml loaded successfully]
[2024-10-20 20:51:14,118:INFO:common:yaml file: C:\Users\Ivar\Downloads\text summarzre\text-summarizer\params.yaml loaded successfully]
[2024-10-20 20:51:14,120:INFO:common:directory: artifacts created]
[2024-10-20 20:51:14,122:INFO:common:directory: artifacts/data_trainer created]


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\Ivar\Downloads\text summarzre\text-summarizer\texts\Lib\site-packages\transformers\training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
  2%|▏         | 1/51 [03:40<3:01:44, 218.08s/it]